In [1]:
# 1. 크롤링 수집된 문서
# 2. 토크나이징 / 노말라이징
# 3. BoW 문서가 벡터로 표현

In [2]:
from konlpy.corpus import kobill
from konlpy.tag import Komoran
from nltk.tokenize import word_tokenize, sent_tokenize
import re

In [3]:
kobill.fileids()

['1809890.txt',
 '1809891.txt',
 '1809892.txt',
 '1809893.txt',
 '1809894.txt',
 '1809895.txt',
 '1809896.txt',
 '1809897.txt',
 '1809898.txt',
 '1809899.txt']

In [4]:
def ngram(s, n=2):
    rst = list()
    for i in range(len(s)-(n-1)):
        rst.append(s[i:i+n])
    return rst

In [5]:
ma = Komoran()

In [6]:
V = list()

for f in kobill.fileids():
    d = kobill.open(f).read()
    d = re.sub(r'^\s|\s$', '', re.sub(r'\s+', ' ', d))
    
    V.extend(word_tokenize(d))
    V.extend(ngram(d, 2))
    V.extend(ngram(d, 3))

    V.extend(ma.morphs(d))
    V.extend(['/'.join(t) for t in ma.pos(d)])  # 형태소


In [7]:
len(V), len(set(V))

(134694, 15274)

In [8]:
from collections import Counter

In [9]:
cv = Counter(V)

In [10]:
# zif's rule
# 상위에 있는애들 하위에 있는 애들 자르는데
# moder ie에서는 굳이 저렇게 안하므로
CV = list(set(V))

In [11]:
D = kobill.fileids()

In [13]:
DTM = [[0 for j in range(len(CV))] for i in range(len(D))]

In [15]:
for f in D:
    i = D.index(f)
    d = kobill.open(f).read()
    d = re.sub(r'^\s|\s$', '', re.sub(r'\s+', ' ', d))

    for t in word_tokenize(d)+ngram(d,3)+ngram(d,2)+ma.morphs(d)+['/'.join(t) for t in ma.pos(d)]:
        # term을 찾아야함
        # cv에서 index 찾기
        j = CV.index(t)
        DTM[i][j] = 1

In [16]:
# Query 생성
Q  = '대통령 국민 국회'
rst = list()
# Query 까기
for q in Q.split(): # Query 개수만큼
    k = CV.index(q)
    rst.append(list())  # Qeury 개수만큼 존재해야하므로 세팅

    # bottleneck
    # Query 개수만큼마다 늘어나야하므로
    # 따라서 바꿈 어떻게? TDM의 형태로

    for i,d in enumerate(DTM):   # Document 개수만큼 돌아야함
        for j,t in enumerate(d): # V만큼 돌아야함
            if k == j and t == 1:   # 이때만 rst에 append
                rst[-1].append(i)

In [17]:
rst

[[0, 1, 3, 4, 6, 9], [4, 6, 7], [0, 1, 2, 3, 6, 7, 8]]

In [18]:
candi = set(rst[0])
for r in rst[1:]:
    candi = candi.intersection(set(r))

In [19]:
list(candi), rst
# 0 번째 문서에 3개의 단어가 다 들어가 있다
# 이게 검색하는 원리
# 가장 빠름

([6], [[0, 1, 3, 4, 6, 9], [4, 6, 7], [0, 1, 2, 3, 6, 7, 8]])

In [20]:
# TDM으로 바꿨을 때
TDM = [[0 for j in range(len(D))] for i in range(len(CV))]  # transpose한 것과 같음

In [21]:
for f in D:
    i = D.index(f)

    d = kobill.open(f).read()
    d = re.sub(r'^\s|\s$', '', re.sub(r'\s+', ' ', d))

    for t in word_tokenize(d)+ngram(d,2)+ngram(d,3)+ma.morphs(d) + ['/'.join(t) for t in ma.pos(d)]:
        j = CV.index(t)
        TDM[j][i] = 1

In [23]:
Q = '대통령 국민 국회'

rst = list()

for q in Q.split():
    k = CV.index(q)
    rst.append([i for i, v in enumerate(TDM[k]) if v == 1])
# bottleneck 사라짐
# inverted index의 구조

# TDM이면서 Inverted Index의 구조를 띈 형태
# 이것을 linked list로 만들어보자

In [24]:
Dictionary = dict()
Posting = list()

for f in D:
    i = D.index(f)

    d = kobill.open(f).read()
    d = re.sub(r'^\s|\s$', '', re.sub(r'\s+', ' ', d))

    for t in word_tokenize(d)+ngram(d,2)+ngram(d,3)+ma.morphs(d) + ['/'.join(t) for t in ma.pos(d)]:
        j = CV.index(t)
        if t not in Dictionary:
            Dictionary[t] = len(Posting)
            Posting.append((i, -1))
        else:
            pos = Dictionary[t]
            Dictionary[t] = len(Posting)
            Posting.append((i, pos))

In [25]:
rst = list()
pos = Dictionary['국민']
while pos != -1:
    i, pos = Posting[pos]
    rst.append(i)

In [26]:
from struct import pack, unpack
# file로 바꾸기 위해서 import

In [28]:
pack('i', 0)

b'\x00\x00\x00\x00'

In [29]:
Dictionary = list()

In [31]:
fp = open('posting.dat', 'wb')
    
for f in D:
    i = D.index(f)

    d = kobill.open(f).read()
    d = re.sub(r'^\s|\s$', '', re.sub(r'\s+', ' ', d))

    local = Counter(word_tokenize(d)+ngram(d,2)+ngram(d,3)+ma.morphs(d) + ['/'.join(t) for t in ma.pos(d)])
    # 문서 1개에 대해서, 단어:빈도, 단어:빈도 ...
    for k, v in local.items():
        if k not in Dictionary:
            Dictionary[k] = fp.tell()
            fp.write(pack('iii', i, v, -1))
        else:
            pos = Dictionary[k]
            Dictionary[k] = fp.tell()        
fp.close()

TypeError: list indices must be integers or slices, not str

In [32]:
pos = Dictionary['국민']

fp.open('posting.dat', 'rb')
while pos != -1:
    fp.seek(pos)
    i, freq,npos = unpack('iii', fp.read(12))
    pos = npos
    print(i, freq)
fp.close()

TypeError: list indices must be integers or slices, not str